# Synopsis:
-iteratively load data, normalize it, and save it here .../MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/normalized_hgg

In [1]:
import utils.hgg_utils as hu
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm 
import nibabel as nib
import pathlib



# Func: Normalize 

In [2]:
def normalize_each_modality_by_its_max(a):

    for modality_idx in range( a.shape[-1] ):    
        a[:, :, :, modality_idx] /= a[:, :, :, modality_idx].max()

# Func: Save the normalized files

In [3]:
def save_data( multi_mod, affines_list, mod_paths, destination ):
    
    patient_paths = [ x.parent.stem for x in mod_paths ]
    mods = [x.name for x in mod_paths]
        
    for modality in range(multi_mod.shape[-1]):
        
        new_file_name = "normalized_" + str(mods[modality])
        
        new_patient_folder = destination.joinpath(patient_paths[modality])
        
        if not new_patient_folder.exists():
            new_patient_folder.mkdir()
        
        new_dest = new_patient_folder.joinpath(new_file_name)
                
        a = nib.Nifti1Image( multi_mod[:,:,:,modality], affine=affines_list[modality] )
        
        nib.save( a, new_dest )

# Normalize and Save the files in folder: .../MICCAI_BraTS_2019_Data_Training/MICCAI_BraTS_2019_Data_Training/normalized_hgg

In [4]:
patient_paths = hu.get_each_hgg_folder()

normalized_hgg = patient_paths[0].parent.parent 
normalized_hgg = normalized_hgg.joinpath("normalized_hgg")

#print(normalized_hgg)
if not normalized_hgg.exists():
    normalized_hgg.mkdir()
    
for patient in tqdm(patient_paths):
     
    X, affines, paths = hu.get_a_multimodal_tensor( patient )
    
    normalize_each_modality_by_its_max(X)
    
    save_data(X, affines, paths, normalized_hgg)
